In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24

mpl.rcParams.update({"axes.grid" : True})
import util as yu

ens='cA211.530.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/test/data_merge/NST_a.h5_avgD'
data=yu.load(path)
Ncfg=yu.deepKey(data['2pt'],2).shape[0]
print('Ncfg='+str(Ncfg))

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/test/data_merge/NST_a.h5_all'
data=yu.load(path)
Ncfg=yu.deepKey(data['2pt'],2).shape[0]
print('Ncfg='+str(Ncfg))

# directions

In [ ]:
thre=4

# 001
for opab in data['2pt'].keys():
    opa,opb=opab.split('_')
    g,pt,irrep,occa,l,flaa=opa.split(';'); g,pt,irrep,occb,l,flab=opb.split(';')
    if pt!='0,0,1':
        continue
    for pt2 in ['0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0']:
        opa2=';'.join([g,pt2,irrep,occa,l,flaa]); opb2=';'.join([g,pt2,irrep,occb,l,flab])
        opab2=opa2+'_'+opb2
        
        for diag in data['2pt'][opab].keys():
            t_dat=data['2pt'][opab][diag]
            t_dat2=data['2pt'][opab2][diag]
            for npRI in [np.real,np.imag]:
                mean,err,_=yu.jackknife(npRI(t_dat))
                mean2,err2,_=yu.jackknife(npRI(t_dat2))

                t=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err2[0]**2)); s=np.sum([1 if ele>thre else 0 for ele in t])
                t2=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err[0]**2)); s2=np.sum([1 if ele>thre else 0 for ele in t2])
                t3=np.abs((mean[0]-mean2[0])/np.sqrt(err2[0]**2+err2[0]**2)); s3=np.sum([1 if ele>thre else 0 for ele in t3])
                if s>0.5 or s2>0.5 or s3>0.5: 
                    print(diag,npRI,opab,pt2)
                    print(s,s2,s3)
                    print(t)
                    print(t2)
                    print(t3)
    # break

In [ ]:
opab='g;0,0,1;G1;a;l2;p_g;0,0,1;G1;N0pi1,a;l2;n,pi+'
diag='T'; npRI=np.imag

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=15; irow=0; icol=0 

dat=data['2pt'][opab][diag]
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax),tMean,tErr,color='r')
axs[irow,icol].set_xticks(range(0,tmax,2))  

for rot,color,shift in zip(['0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0'],['g','b','orange','purple','cyan'],[0.1,0.2,0.3,0.4,0.5]):
    opa,opb=opab.split('_')
    opab2=opa.replace('0,0,1',rot)+'_'+opb.replace('0,0,1',rot)
    
    dat=data['2pt'][opab2][diag]
    mean,err,cov=yu.jackknife(npRI(dat))
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax)+shift,tMean,tErr,color=color)
    axs[irow,icol].set_xticks(range(0,tmax,2))  
    # axs[irow,icol].set_ylim(4e-8,5e-8)
    None

In [ ]:
path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_GEVP/templateFiles/templateMomList.h5'
with h5py.File(path) as f:
    moms=f['D1ii'][:]
    dic={}
    for i,mom in enumerate(moms):
        dic[tuple(mom)]=i
    print(f['D1ii'])
    
    i_mom=dic[(0,0,0, 0,0,0, 0,0,1)]
    
    i_mom2=dic[(0,0,0, 0,0,0, 0,1,0)]
    
npRI=np.real

fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/NJNpi_GEVP/avg-merge/N_2pt-pi0f.h5'
with h5py.File(path) as f:
    cfgs=list(f.keys())
    cfg=cfgs[0]
    dat=np.array([f[cfg]['12']['pi2=0_0_0']['p_p'][:,i_mom,0,0*4+0] for cfg in cfgs])
    
    mean,err,cov=yu.jackknife(-npRI(dat))
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='r')
    axs[irow,icol].set_xticks(range(0,tmax,2))  
    
    dat=(np.array([f[cfg]['12']['pi2=0_0_0']['p_p'][:,i_mom2,0,0*4+0] for cfg in cfgs])
    -1j*np.array([f[cfg]['12']['pi2=0_0_0']['p_p'][:,i_mom2,0,1*4+0] for cfg in cfgs])
    +1j*np.array([f[cfg]['12']['pi2=0_0_0']['p_p'][:,i_mom2,0,0*4+1] for cfg in cfgs])
    +np.array([f[cfg]['12']['pi2=0_0_0']['p_p'][:,i_mom2,0,1*4+1] for cfg in cfgs]))/2
    mean,err,cov=yu.jackknife(-npRI(dat))
    tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
    axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
    axs[irow,icol].set_xticks(range(0,tmax,2))  

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/obs/test_debug/data_N-pi0/'
cfgs=os.listdir(path)
data=[]
for cfg in cfgs:
    infile=path+cfg+'/N-pi0f.h5'
    with h5py.File(infile) as f:
        data.append(f['data'][:])

data=np.array(data)
data.shape

In [ ]:
path='/project/s1174/lyan/code/projectData/NST_a/obs/test_debug/data_N-pi0/a0000/N-pi0f.h5'
with h5py.File(path) as f:
    moms=f['moms'][:]
    for i,mom in enumerate(moms):
        dic[tuple(mom)]=i
    i_mom=dic[(0,0,0, 0,0,0, 0,0,1, 0,0,0)]
    i_mom2=dic[(0,0,0, 0,0,0, 0,1,0, 0,0,0)]

i_fla=0

fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

dat=data[:,i_fla,:16,i_mom,0*4+0]
mean,err,cov=yu.jackknife(-npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='r')
axs[irow,icol].set_xticks(range(0,tmax,2))  

dat=(data[:,i_fla,:16,i_mom2,0*4+0]
-1j*data[:,i_fla,:16,i_mom2,1*4+0]
+1j*data[:,i_fla,:16,i_mom2,0*4+1]
+data[:,i_fla,:16,i_mom2,1*4+1])/2
mean,err,cov=yu.jackknife(-npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
axs[irow,icol].set_xticks(range(0,tmax,2))  

None

# Direction for 3pt

In [ ]:
import pickle
with open('aux/avgDirection.pkl','rb') as f:
    avgD=pickle.load(f)

def rotate(opabj,rot):
    temp=[] # rotate insert
    for coe,opab,insert in opabj:
        gm,j,tf=insert.split('_')
        if gm in ['id','gt','g5','g5gt']:
            temp.append((coe,opab,insert))
        elif gm in ['gx','gy','gz']:
            i_gm={'gx':0,'gy':1,'gz':2}[gm]
            for j_gm,val in enumerate(avgD[rot]['gamma_i'][i_gm,:]):
                if np.abs(val)<1e-7:
                    continue
                insert_new='_'.join([['gx','gy','gz'][j_gm],j,tf])
                temp.append((coe*val,opab,insert_new))
        elif gm in ['g5gx','g5gy','g5gz']:
            i_gm={'g5gx':0,'g5gy':1,'g5gz':2}[gm]
            for j_gm,val in enumerate(avgD[rot]['gamma_i'][i_gm,:]):
                if np.abs(val)<1e-7:
                    continue
                insert_new='_'.join([['g5gx','g5gy','g5gz'][j_gm],j,tf])
                temp.append((coe*val,opab,insert_new))
        else:
            1/0
            
    temp2=[] # rotate opa
    for coe,opab,insert in temp:
        opa,opb=opab.split('_')
        g,pt,irrep,occ,lam,fla=opa.split(';')
        assert(pt in ['0,0,0','0,0,1'])
        if pt !='0,0,0':
            opa_new=';'.join([g,rot,irrep,occ,lam,fla])
            temp2.append((coe,'_'.join([opa_new,opb]),insert))
        else:
            i_lam={'l1':0,'l2':1}[lam]
            for j_lam,val in enumerate(avgD[rot]['irrep_row'][i_lam,:]):
                if np.abs(val)<1e-7:
                    continue
                opa_new=';'.join([g,pt,irrep,occ,['l1','l2'][j_lam],fla])
                temp2.append((coe*val,'_'.join([opa_new,opb]),insert))
                
    temp3=[] # rotate opb
    for coe,opab,insert in temp2:
        opa,opb=opab.split('_')
        g,pt,irrep,occ,lam,fla=opb.split(';')
        assert(pt in ['0,0,0','0,0,1'])
        if pt !='0,0,0':
            opb_new=';'.join([g,rot,irrep,occ,lam,fla])
            temp3.append((coe,'_'.join([opa,opb_new]),insert))
        else:
            i_lam={'l1':0,'l2':1}[lam]
            for j_lam,val in enumerate(avgD[rot]['irrep_row'][i_lam,:]):
                if np.abs(val)<1e-7:
                    continue
                opb_new=';'.join([g,pt,irrep,occ,['l1','l2'][j_lam],fla])
                temp3.append((coe*np.conj(val),'_'.join([opa,opb_new]),insert))

    return temp3

def getDat(opabj,diag):
    return np.sum([data['3pt'][opab][insert][diag]*coe  for coe,opab,insert in opabj],axis=0)

getDat([(1,'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;a;l1;p','g5gy_j+_10')],'NJN').shape

In [ ]:
# thre=4

# opabs=[]
# for i_opab,opab in enumerate(data['3pt'].keys()):
#     opa,opb=opab.split('_')
#     _,pta,_,_,_,_=opa.split(';'); _,ptb,_,_,_,_=opb.split(';')
#     if (pta not in ['0,0,0','0,0,1']) or (ptb not in ['0,0,0','0,0,1']):
#         continue
#     opabs.append(opab)

# for i_opab,opab in enumerate(opabs):
#     opa,opb=opab.split('_')
#     _,pta,_,_,_,_=opa.split(';'); _,ptb,_,_,_,_=opb.split(';')
#     if (pta not in ['0,0,0','0,0,1']) or (ptb not in ['0,0,0','0,0,1']):
#         continue
#     for i_insert,insert in enumerate(data['3pt'][opab].keys()):
#         gm,j,tf=insert.split('_')
#         if j!='j+' or tf!='10':
#             continue    
#         opabj=[(1,opab,insert)]
#         for diag in data['3pt'][opab][insert].keys():
#             for rot in ['0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0']:
#                 t_dat=getDat(opabj,diag)
#                 t_dat2=getDat(rotate(opabj,rot),diag)
#                 for npRI in [np.real,np.imag]:
#                     mean,err,_=yu.jackknife(npRI(t_dat))
#                     mean2,err2,_=yu.jackknife(npRI(t_dat2))

#                     t=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err2[0]**2)); s=np.sum([1 if ele>thre else 0 for ele in t])
#                     t2=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err[0]**2)); s2=np.sum([1 if ele>thre else 0 for ele in t2])
#                     t3=np.abs((mean[0]-mean2[0])/np.sqrt(err2[0]**2+err2[0]**2)); s3=np.sum([1 if ele>thre else 0 for ele in t3])
#                     if s>0.5 or s2>0.5 or s3>0.5: 
#                         print(opabj,diag,rot,npRI)
#                         print(rotate(opabj,rot))
#                         print(s,s2,s3)
#                         print(t)
#                         print(t2)
#                         print(t3)
#         print(i_opab,len(opabs),i_insert,len(data['3pt'][opab].keys()))
                        
#     # break

In [ ]:
tfs=[10,12,14]
def rep_tf(opabj,tf_new):
    return [(coe,opab,insert[:-2]+str(tf_new)) for coe,opab,insert in opabj]


opabj=[(1, 'g;0,0,0;G1g;N1pi1,a;l1;p,pi0_g;0,0,1;G1;a;l2;p', 'g5gt_j+_10')] 
diag='N-jPf'; npRI=1


fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

def func(dat):
    t=yu.meanDeep(dat)
    t=[t[i] for i,tf in enumerate(tfs)]
    t=yu.prefactorDeep(t,npRI)
    return t

dat=[getDat(rep_tf(opabj,tf),diag)[:,1:tf] for tf in tfs]

(mean,err,cov)=yu.jackknife(dat,func)

for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    # print(tMean)
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1))

for rot,color,shift in zip(['0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0'],['g','b','orange','purple','cyan'],[0.1,0.2,0.3,0.4,0.5]):
    dat=[getDat(rotate(rep_tf(opabj,tf),rot),diag)[:,1:tf] for tf in tfs]

    (mean,err,cov)=yu.jackknife(dat,func)

    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+shift,tMean,tErr,color=color,fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))

    None


In [ ]:
tfs=[10,12,14]
def rep_tf(opabj,tf_new):
    return [(coe,opab,insert[:-2]+str(tf_new)) for coe,opab,insert in opabj]

# good
opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;a;l1;p', 'id_j+_10')]
diag='NJN'; npRI=1
opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;a;l1;p', 'g5_j+_10')]
diag='N-j'; npRI=1
# bad
opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;a;l1;p', 'g5_j+_10')]
diag='NJN'; npRI=1

# good
opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'id_j+_10')]
diag='N-jPi'; npRI=-1j
opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'id_j+_10')]
diag='T-j'; npRI=-1j
# bad
opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'id_j+_10')]
diag='NJN-pi0i'; npRI=1


# opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'g5_j-_10')]
# diag='T-j'; npRI=-1j

opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'g5gz_j+_10')]
diag='Z3pt'; npRI=1



fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

def func(dat):
    t=yu.meanDeep(dat)
    t=[t[i] for i,tf in enumerate(tfs)]
    t=yu.prefactorDeep(t,npRI)
    return t

dat=[getDat(rep_tf(opabj,tf),diag)[:,1:tf] for tf in tfs]

(mean,err,cov)=yu.jackknife(dat,func)

for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    # print(tMean)
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1))

for rot,color,shift in zip(['0,0,-1','0,1,0','0,-1,0','1,0,0','-1,0,0'],['g','b','orange','purple','cyan'],[0.1,0.2,0.3,0.4,0.5]):
    dat=[getDat(rotate(rep_tf(opabj,tf),rot),diag)[:,1:tf] for tf in tfs]

    (mean,err,cov)=yu.jackknife(dat,func)

    for i_tf,tf in enumerate(tfs):
        tMean=mean[i_tf];tErr=err[i_tf]
        # print(tMean)
        axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+shift,tMean,tErr,color=color,fmt=['s','d','o'][i_tf])
    axs[irow,icol].set_xticks(range(-6,7,1))

    None


In [ ]:
def cfg2out(cfg):
    path='/project/s1174/fpittler/code/projectData/nucleon_sigma_term/cA211'+cfg[0]+'.53.24/NJN/outputdata_sinkMom_GOOD/'+cfg[1:]+'/'
    return path

with open('/project/s1174/lyan/code/projectData/NST_a/test/data_aux/cfgs_run') as f:
    cfgs=f.read().splitlines()

flag=True
dat=[]
for i,cfg in enumerate(cfgs):
    inpath=cfg2out(cfg)
    files=[file for file in os.listdir(inpath) if file.endswith('dt10_protonup.h5')]
    with h5py.File(inpath+files[0]) as f:
        src=list(f.keys())[0]
        if flag:
            moms=f[src]['mvec'][:]
            flag=False
        t=f[src]['MprotonUp'][()]
        dat.append(t)
    print(i,len(cfgs),end='\r')
dat=np.array(dat)
dat.shape

In [ ]:
import sympy as sp
gamma_1=sp.Matrix([[0.,0.,0.,1j],[0.,0.,1j,0.],[0.,-1j,0.,0.],[-1j,0.,0.,0.]])
gamma_2=sp.Matrix([[0.,0.,0.,1.],[0.,0.,-1.,0.],[0.,-1.,0.,0.],[1.,0.,0.,0.]])
gamma_3=sp.Matrix([[0.,0.,1j,0.],[0.,0.,0.,-1j],[-1j,0.,0.,0.],[0.,1j,0.,0.]])
gamma_4=sp.Matrix([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,-1.,0.],[0.,0.,0.,-1.]])

id=(gamma_4*gamma_4)
gamma_5=(gamma_1*gamma_2*gamma_3*gamma_4).applyfunc(sp.nsimplify)

dic={}
for i,mom in enumerate(moms):
    dic[tuple(mom)]=i
    
i_mom=dic[(0,0,0, 0,0,0, 0,0,0, 0,0,1)]
i_gm=3
npRI=np.imag
    
t=dat[...,0]+1j*dat[...,1]
t=t[:,:,i_mom,i_gm,:]
t=np.reshape(t,t.shape[:2]+(4,4))
t=np.einsum('ctab,ba->ct',t,np.array((id+gamma_4)*(1j*gamma_5*gamma_3)/2,dtype=complex))
t.shape

(mean,err,cov)=yu.jackknife(npRI(t))
np.array([mean[0],err[0]]).T

In [ ]:
(id+gamma_4)*(1j*gamma_5*gamma_3)/2

In [ ]:
tfs=[10,12,14]
def rep_tf(opabj,tf_new):
    return [(coe,opab,insert[:-2]+str(tf_new)) for coe,opab,insert in opabj]

# opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'id_j+_10')]
# diag='N-jPi'; rot='0,1,0'; npRI=-1j

opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'id_j+_10')]
diag='NJN-pi0i'; rot='0,1,0'; npRI=1
# diag='N-jPi'; rot='0,1,0'; npRI=-1j

opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'g5gz_j-_10')]
diag='N-jPi'; rot='0,1,0'; npRI=-1j

# opabj=[(1, 'g;0,0,0;G1g;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0', 'g5_j-_10')]
# diag='N-jPi'; rot='0,1,0'; npRI=1

fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

def func(dat):
    t=yu.meanDeep(dat)
    t=[t[i] for i,tf in enumerate(tfs)]
    t=yu.prefactorDeep(t,npRI)
    return t

dat=[getDat(rep_tf(opabj,tf),diag)[:,1:tf] for tf in tfs]

(mean,err,cov)=yu.jackknife(dat,func)

for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    # print(tMean)
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)-0.1,tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1))

dat=[getDat(rotate(rep_tf(opabj,tf),rot),diag)[:,1:tf] for tf in tfs]

(mean,err,cov)=yu.jackknife(dat,func)

for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    # print(tMean)
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+0.1,tMean,tErr,color='b',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1))

None


# N-pi0i vs. N-pi0f

In [ ]:
Ncfg=-1

for opab in data['2pt'].keys():
    t_dat=data['2pt'][opab]
    diag='N-pi0i'
    if diag not in t_dat.keys():
        continue
    opa,opb=opab.split('_')
    t_dat2=data['2pt'][opb+'_'+opa]
    diag2='N-pi0f'
    for npRI in [np.real,np.imag]:
        mean,err,_=yu.jackknife(npRI(t_dat[diag])[:Ncfg])
        mean2,err2,_=yu.jackknife(npRI(t_dat2[diag2][:Ncfg])* (-1 if npRI==np.imag else 1))

        t=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err2[0]**2))
        t2=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err[0]**2))
        t3=np.abs((mean[0]-mean2[0])/np.sqrt(err2[0]**2+err2[0]**2))
        if np.max([t,t2,t3])>(4*np.sqrt(2)): # sqrt(2) for anti correlation
            print(diag,npRI,opab)
            print(t)
            # print(t2)
            # print(t3)

In [ ]:
Ncfg=0; NcfgMax=-1

opab='g;0,0,1;G1;a;l1;p_g;0,0,1;G1;N1pi0,a;l1;p,pi0'
diag='N-pi0i'; diag2='N-pi0f'; npRI=np.real

opa,opb=opab.split('_')

fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

dat=data['2pt'][opab][diag]
mean,err,cov=yu.jackknife(npRI(dat)[Ncfg:NcfgMax])
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='r')
axs[irow,icol].set_xticks(range(0,tmax,2))  

dat=data['2pt'][opb+'_'+opa][diag2]
mean,err,cov=yu.jackknife(npRI(dat)[Ncfg:NcfgMax]*(-1 if npRI==np.imag else 1))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
axs[irow,icol].set_xticks(range(0,tmax,2))  

dat=(data['2pt'][opab][diag]+data['2pt'][opb+'_'+opa][diag2])/2
mean,err,cov=yu.jackknife(npRI(dat)[Ncfg:NcfgMax])
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='g')
axs[irow,icol].set_xticks(range(0,tmax,2))  

None

# N-j-pi0i vs. N-pi0f-j

In [ ]:
diag='N-j-pi0i'; diag2='N-pi0f-j'; thre=4*np.sqrt(2) # sqrt(2) for anti correlation
diag='N-jPi'; diag2='N-jPf'; thre=4

Ncfg=-1

conjList=['id','gt','g5gt']

opabs=data['3pt'].keys()
for i,opab in enumerate(opabs):
    for insert in data['3pt'][opab].keys(): 
        t_dat=data['3pt'][opab][insert]
        # diag='N-j-pi0i'
        if diag not in t_dat.keys():
            continue
        opa,opb=opab.split('_')
        t_dat2=data['3pt'][opb+'_'+opa][insert]
        # diag2='N-pi0f-j'
        
        gm,j,tf=insert.split('_')
        if j!='j+' or tf != '10':
            continue
        conjSgn=1 if gm in conjList else -1
        
        for npRI in [np.real,np.imag]:
            mean,err,_=yu.jackknife(npRI(t_dat[diag])[:Ncfg])
            mean2,err2,_=yu.jackknife(npRI(t_dat2[diag2][:Ncfg][:,::-1])* (-1 if npRI==np.imag else 1) * conjSgn)
            
            t=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err2[0]**2)); s=np.sum([1 if ele>thre else 0 for ele in t])
            t2=np.abs((mean[0]-mean2[0])/np.sqrt(err[0]**2+err[0]**2)); s2=np.sum([1 if ele>thre else 0 for ele in t2])
            t3=np.abs((mean[0]-mean2[0])/np.sqrt(err2[0]**2+err2[0]**2)); s3=np.sum([1 if ele>thre else 0 for ele in t3])
            if s>0.5 or s2>0.5 or s3>0.5: 
                print(diag,npRI,opab,insert)
                print(s,s2,s3)
                print(t)
                print(t2)
                print(t3)
    print(str(i)+'/'+str(len(opabs)),end='\r')

In [ ]:
diag='N-j-pi0i'; diag2='N-pi0f-j'; thre=4*np.sqrt(2) # sqrt(2) for anti correlation
diag='N-jPi'; diag2='N-jPf'; thre=4

opab='g;0,0,0;G1g;a;l1;n_g;0,0,1;G1;N1pi0,a;l1;n,pi0'
insert='id_j-'; prefactor=1

tfs=[10,12,14]
gm,j=insert.split('_')
opa,opb=opab.split('_')
conjSgn=1 if gm in conjList else -1

fig, axs = yu.getFigAxs(1,1)

t_dat=[data['3pt'][opab][insert+'_'+str(tf)][diag][:,1:tf] for tf in tfs]
def func(dat):
    t=yu.meanDeep(dat)
    t=yu.prefactorDeep(t,prefactor)
    return t
(mean,err,cov)=yu.jackknife(t_dat,func)
irow=0; icol=0
for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1))

t_dat=[data['3pt'][opb+'_'+opa][insert+'_'+str(tf)][diag2][:,1:tf][:,::-1] for tf in tfs]
def func(dat):
    t=yu.meanDeep(dat)
    t=yu.prefactorDeep(t,prefactor*conjSgn*(-1 if prefactor==-1j else 1))
    return t
(mean,err,cov)=yu.jackknife(t_dat,func)
irow=0; icol=0
for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='b',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1));

t_dat=[(data['3pt'][opab][insert+'_'+str(tf)][diag][:,1:tf]+
       data['3pt'][opb+'_'+opa][insert+'_'+str(tf)][diag2][:,1:tf][:,::-1]*conjSgn*(-1 if prefactor==-1j else 1))/2 for tf in tfs]
def func(dat):
    t=yu.meanDeep(dat)
    t=yu.prefactorDeep(t,prefactor)
    return t
(mean,err,cov)=yu.jackknife(t_dat,func)
irow=0; icol=0
for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='g',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1));

# 2pt fw vs. bw

In [ ]:
for opab in data['2pt'].keys():
    t_dat=data['2pt'][opab]
    for diag_bw in t_dat.keys():
        if '_bw' not in diag_bw:
            continue
        diag=diag_bw.replace('_bw','')
        if diag in ['W2pt']:
            continue
        for npRI in [np.real,np.imag]:
            mean,err,_=yu.jackknife(npRI(t_dat[diag]))
            mean_bw,err_bw,_=yu.jackknife(npRI(t_dat[diag_bw]))

            t=np.abs((mean[0]-mean_bw[0])/np.sqrt(err[0]**2+err_bw[0]**2))
            t2=np.abs((mean[0]-mean_bw[0])/np.sqrt(err[0]**2+err[0]**2))
            t3=np.abs((mean[0]-mean_bw[0])/np.sqrt(err_bw[0]**2+err_bw[0]**2))
            if np.max([t,t2,t3])>4:
                print(diag,npRI,opab)
                print(t)
    # break

In [ ]:
# opab='g;0,0,1;G1;N1pi0,a;l1;p,pi0_g;0,0,1;G1;N0pi1,a;l1;n,pi+'
opab='g;0,0,-1;G1;N1pi0,a;l1;p,pi0_g;0,0,-1;G1;a;l1;p'
diag='N-pi0f'; diag_bw='N_bw-pi0f'; npRI=np.real

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=15; irow=0; icol=0

dat=data['2pt'][opab][diag]
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='r')
axs[irow,icol].set_xticks(range(0,tmax,2))  

dat=data['2pt'][opab][diag_bw]
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
axs[irow,icol].set_xticks(range(0,tmax,2))  
None

In [ ]:
data['VEV']['pi0f'].shape


In [ ]:
opab='g;0,0,-1;G1;N0pi1,a;l1;p,pi0_g;0,0,-1;G1;N0pi1,a;l1;p,pi0'
# opab='g;0,0,0;G1g;N1pi1,a;l2;p,pi0_g;0,0,0;G1g;N1pi1,a;l2;n,pi+'
diag='T-pi0f'; diag_bw='T_bw-pi0f'; npRI=np.real
# diag='N-pi0f-pi0i'; diag_bw='N_bw-pi0f-pi0i'; npRI=np.real

# opab='g;0,1,0;G1;N1pi0,a;l1;n,pi+_g;0,1,0;G1;N0pi1,a;l1;n,pi+'
# diag='N-P'; diag_bw='N_bw-P'; npRI=np.imag

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=15; irow=0; icol=0

opab_t='g;0,0,-1;G1;a;l1;p_g;0,0,-1;G1;N0pi1,a;l1;p,pi0'

dat=data['2pt'][opab][diag]
print(dat.shape)
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='r')
axs[irow,icol].set_xticks(range(0,tmax,2))  

dat=data['2pt'][opab][diag_bw]
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
axs[irow,icol].set_xticks(range(0,tmax,2))  
None

In [ ]:
opab='g;0,0,1;G1;N1pi0,a;l1;p,pi0_g;0,0,1;G1;N0pi1,a;l1;p,pi0'
# opab='g;0,0,0;G1g;N1pi1,a;l2;p,pi0_g;0,0,0;G1g;N1pi1,a;l2;n,pi+'
diag='T-pi0f'; diag_bw='T_bw-pi0f'; npRI=np.real
# diag='N-pi0f-pi0i'; diag_bw='N_bw-pi0f-pi0i'; npRI=np.real

# opab='g;0,1,0;G1;N1pi0,a;l1;n,pi+_g;0,1,0;G1;N0pi1,a;l1;n,pi+'
# diag='N-P'; diag_bw='N_bw-P'; npRI=np.imag

fig, axs = yu.getFigAxs(1,1)
tmin=0; tmax=15; irow=0; icol=0

opab_t='g;0,0,1;G1;a;l1;p_g;0,0,1;G1;N0pi1,a;l1;p,pi0'

dat=data['2pt'][opab][diag]-data['2pt'][opab_t]['T']*data['VEV']['pi0f'][:,None]
print(dat.shape)
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)-0.1,tMean,tErr,color='r')
axs[irow,icol].set_xticks(range(0,tmax,2))  

dat=data['2pt'][opab][diag_bw]#-data['2pt'][opab_t]['T_bw']*data['VEV']['pi0f'][:,None]
mean,err,cov=yu.jackknife(npRI(dat))
tMean=mean[0][tmin:tmax]; tErr=err[0][tmin:tmax]
axs[irow,icol].errorbar(np.arange(tmin,tmax)+0.1,tMean,tErr,color='b')
axs[irow,icol].set_xticks(range(0,tmax,2))  
None

# 3pt fw vs. bw

In [ ]:
for i,opab in enumerate(data['3pt'].keys()):
    print(i,len(data['3pt'].keys()),end='      \r')
    for insert in data['3pt'][opab].keys():
        gm,j,tf=insert.split('_')
        if j!='j+' or tf != '10':
            continue
        
        t_dat=data['3pt'][opab][insert]
        
        for diag_bw in t_dat.keys():
            if '_bw' not in diag_bw:
                continue
            diag=diag_bw.replace('_bw','')
            if diag=='T-j':
                continue
            # if diag in ['W2pt']:
            #     continue
            for npRI in [np.real,np.imag]:
                mean,err,_=yu.jackknife(npRI(t_dat[diag]))
                mean_bw,err_bw,_=yu.jackknife(npRI(t_dat[diag_bw]))

                t=np.abs((mean[0]-mean_bw[0])/np.sqrt(err[0]**2+err_bw[0]**2))
                t2=np.abs((mean[0]-mean_bw[0])/np.sqrt(err[0]**2+err[0]**2))
                t3=np.abs((mean[0]-mean_bw[0])/np.sqrt(err_bw[0]**2+err_bw[0]**2))
                if np.max([t,t2,t3])>4:
                    print(diag,npRI,opab,insert)
                    print(t)

In [ ]:
tfs=[10,12,14]

opab='g;0,0,0;G1;a;l1;p_g;0,0,1;G1;a;l1;p'; insert='id_j+'
diag='N-j'; npRI=-1j

t=diag.split('-')
t[0]=t[0]+'_bw'
diag2='-'.join(t)

fig, axs = yu.getFigAxs(1,1)
tmin=1; tmax=15; irow=0; icol=0 

def func(dat):
    t=yu.meanDeep(dat)
    t=[t[i] for i,tf in enumerate(tfs)]
    t=yu.prefactorDeep(t,npRI)
    return t

dat=[data['3pt'][opab][insert+'_'+str(tf)][diag][:,1:tf] for tf in tfs]

(mean,err,cov)=yu.jackknife(dat,func)

for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    # print(tMean)
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='r',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1));

dat=[data['3pt'][opab][insert+'_'+str(tf)][diag2][:,1:tf] for tf in tfs]

(mean,err,cov)=yu.jackknife(dat,func)

for i_tf,tf in enumerate(tfs):
    tMean=mean[i_tf];tErr=err[i_tf]
    # print(tMean)
    axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,color='b',fmt=['s','d','o'][i_tf])
axs[irow,icol].set_xticks(range(-6,7,1));